# Creating a RAG model with Langchain and Huggingface

Kelompok 1 Algoritma Deep Learning
Anggota Kelompok :
1. A IAS Falah Surya Gemilang
2. Aldo Rizky Ramadhan
3. Faiz Rizki Azmi
4. Muhamad Ibnu Khaidar Hafiz
5. Siti Asma Tomu

In [1]:
## installs
%pip install -qU langchain-nvidia-ai-endpoints

In [2]:
from google.colab import userdata
import os

os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_TOKEN')

from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [3]:
%pip install -q langchain_community
%pip install -q langchain_chroma
%pip install -q langchain

In [19]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_nvidia import NVIDIAEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=NVIDIAEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is 3 common choices for ANN Algorithm for fast MIPS?")
# rag_chain.invoke("What is Task Decomposition?")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Based on the provided context, three common choices for ANN Algorithm for fast MIPS are not explicitly mentioned. However, Fig. 9 mentions a comparison of MIPS algorithms, and the website ann-benchmarks.com provides more information on MIPS algorithms and their performance comparison.'

In [18]:
# cleanup
vectorstore.delete_collection()